In [43]:
import pandas as pd

In [44]:
data =pd.read_csv("./data/data_bank.csv")

In [46]:
data.head(5)

,language,url,date,content
0,en,https://www.ecb.europa.eu/pub/pdf/other/SESFOD...,2 February 2024,Error
1,en,https://www.ecb.europa.eu/stats/ecb_surveys/ba...,23 January 2024,Search in this publication\nIn the January 202...
2,en,https://www.ecb.europa.eu/stats/ecb_surveys/ba...,24 October 2023,Search in this publication\nThe results report...
3,en,https://www.ecb.europa.eu/pub/pdf/other/SESFOD...,6 October 2023,Error
4,en,https://www.ecb.europa.eu/stats/ecb_surveys/ba...,25 July 2023,Search in this publication\nThe results report...


In [ ]:
# Error c est quand l'article c'est un pdf (pas scrappable)

In [54]:
print(data["url"].iloc[2])

https://www.ecb.europa.eu/stats/ecb_surveys/bank_lending_survey/html/ecb.blsurvey2023q3~b960111b2d.en.html


In [53]:
print(data["content"].iloc[2][:5000])
print("\nTaille de l'article (nombre de caractères):\n",len(data["content"].iloc[2]))

Search in this publication
The results reported in the October 2023 bank lending survey (BLS) relate to changes observed during the third quarter of 2023 and expectations for the fourth quarter of 2023. The survey was conducted between 15 September and 2 October 2023.A total of 157 banks were surveyed in this round, with a response rate of 100%. In addition to results for the euro area as a whole, this report contains results for the four largest euro area countries.[1]
A number of ad hoc questions were included in the October 2023 survey. They address the impact of the situation in financial markets on banks’ access to retail and wholesale funding, the impact of the ECB’s monetary policy asset portfolio and TLTRO III on banks and their lending policies, and the impact of key ECB interest rate decisions on bank profitability.
In the October 2023 BLS, euro area banks indicated that their credit standards for loans or credit lines to enterprises tightened further in the third quarter of 